In [1]:
import sys
from pathlib import Path

repo_dir = Path('..').resolve()
assert repo_dir.name == 'hicetnunc-dataset', repo_dir
if str(repo_dir) not in sys.path:
    sys.path.append(str(repo_dir))

import src.reload; src.reload.reload()

In [165]:
# accounts_metadata = {}
# accounts_metadata = src.utils.read_json(src.config.tzktio_accounts_metadata_file)

In [166]:
from tqdm import tqdm
import requests

for addr in tqdm(addrs_db):
    if addr in accounts_metadata:
        continue

    req = requests.get(f'https://api.tzkt.io/v1/accounts/{addr}/metadata')
    if req.status_code == 200:
        accounts_metadata[addr] = req.json()
    elif req.status_code == 204:
        accounts_metadata[addr] = None
    else:
        print(addr, req.status_code, req.text)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3753/3753 [17:04<00:00,  3.66it/s]


In [170]:
from collections import Counter

addr_meta_fields_cnt = Counter()
examples = {}

for addr, addr_meta in accounts_metadata.items():
    if addr_meta is None:
        continue
    for field in addr_meta:
        addr_meta_fields_cnt[field] += 1
        if len(addr_meta[field]) < 15:
            examples[field] = addr_meta[field]

print(len(accounts_metadata))

# https://api.tzkt.io/#operation/Accounts_GetMetadata
addr_meta_fields_cnt.most_common()

3753


[('kind', 676),
 ('alias', 676),
 ('logo', 674),
 ('twitter', 652),
 ('description', 628),
 ('email', 477),
 ('site', 416),
 ('instagram', 399),
 ('address', 306),
 ('github', 114),
 ('telegram', 60),
 ('facebook', 52),
 ('reddit', 20),
 ('owner', 1)]

In [27]:
%run -i ../src/reload.py

In [172]:
src.utils.write_json(addrs_db, src.config.addrs_db_json_file)

written 1186084 bytes 3753 entries
